In [ ]:
import pickle
import pandas as pd

# Inspect some matches

## Load and see how many candidates there are

In [ ]:
df = pd.read_pickle("../scripts/indexed.pickled")

In [ ]:
lookup = {}
for t in df.itertuples():
    lookup[t.Index] = t

In [ ]:
del(df)

In [ ]:
matches = pd.read_csv("m-JKP.csv", delimiter="|")

In [ ]:
len(matches)

## Uniqueify by taking max both ways

In [ ]:
# for computing upper bound on number of extracted matches
g1 = matches.groupby(by=["a_FT", "a_Kipnr", "a_Løbenr"], as_index=False)
g2 = matches.groupby(by=["b_FT", "b_Kipnr", "b_Løbenr"])
l1=list(g1.size().groupby(level=0).size().iteritems())
l2=list(g2.size().groupby(level=0).size().iteritems())
print(l1)
print(l2)

In [ ]:
g1 = matches.groupby(by=["a_FT", "a_Kipnr", "a_Løbenr"])
idx = g1["p"].transform(max) == matches["p"]
matches = matches[idx]
print(len(matches))

In [ ]:
g2 = matches.groupby(by=["b_FT", "b_Kipnr", "b_Løbenr"])
idx = g2["p"].transform(max) == matches["p"]
matches = matches[idx]
print(len(matches))

In [ ]:
len(matches)

In [ ]:
for (a_year, a_count), (b_year, b_count) in zip(l1, l2):
    print(a_year, b_year, min(a_count, b_count))
print(sum(min(a, b) for ((_, a), (_, b)) in zip(l1,l2)))

In [ ]:
# so how many left?
g1 = matches.groupby(by=["a_FT", "a_Kipnr", "a_Løbenr"], as_index=False)
g2 = matches.groupby(by=["b_FT", "b_Kipnr", "b_Løbenr"])
l1=list(g1.size().groupby(level=0).size().iteritems())
l2=list(g2.size().groupby(level=0).size().iteritems())

In [ ]:
for (a_year, a_count), (b_year, b_count) in zip(l1, l2):
    print(a_year, b_year, min(a_count, b_count))
print(sum(min(a, b) for ((_, a), (_, b)) in zip(l1,l2)))

## There are probably still duplicates
Obvious if `len(matches)` above is greater than the sum in the cell directly above.

In [ ]:
indexed = matches.set_index(["a_FT", "a_Kipnr", "a_Løbenr"])

In [ ]:
d = indexed.index.get_duplicates()

In [ ]:
indexed.loc[d]

In [ ]:
len(matches)

In [ ]:
matches.drop_duplicates(subset=["a_FT", "a_Løbenr", "a_Kipnr"], inplace=True)
matches.drop_duplicates(subset=["b_FT", "b_Løbenr", "b_Kipnr"], inplace=True)

In [ ]:
len(matches)

In [ ]:
matches

## Now only sensible matches left
So let's look at these

In [ ]:
header = "_|Amt|Herred|Sogn|Navn|Køn|Fødested|Fødeår|Civilstand|Position|Erhverv|Fornavn|Kipnr|Løbenr|Group".split("|")

In [ ]:
for _, a_FT, a_Kip, a_Løb, b_FT, b_Kip, b_Løb, prob in matches.itertuples():
    a_key = (a_FT, a_Kip, a_Løb)
    b_key = (b_FT, b_Kip, b_Løb)
    a = lookup[a_key]
    b = lookup[b_key]
    print("FT", a_FT, "vs", b_FT)
    print(prob)
    for i in range(1, 12):
        s = "{:10s} {:40s} {:40s}".format(header[i], str(a[i]), str(b[i]))
        if a[i] != b[i]:
            s = '\033[95m' + s + "\033[0m"
        print(s)
    print()